In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS']='1'

In [2]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub

2025-04-19 16:40:54.442850: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 16:40:54.586202: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 16:40:54.729512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745073654.880043    7175 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745073654.915625    7175 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745073655.256417    7175 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
import pandas as pd

df = pd.read_csv("data/spam.csv", encoding='iso-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [4]:
df = df.iloc[:,0:2]

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df = df.rename(columns={'v1': 'category', 'v2': 'message'})

In [7]:
df.groupby("category").describe()

message                                                            \
           count unique                                                top   
category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
category       
ham        30  
spam        4

In [8]:
df['spam'] = df['category'].apply(lambda x: 1 if x =='spam' else 0)
df.head()

,category,message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['message'],df['spam'],test_size=0.2,stratify=df['spam'])

In [10]:
y_train.value_counts()

spam
0    3859
1     598
Name: count, dtype: int64

In [11]:
y_test.value_counts()

spam
0    966
1    149
Name: count, dtype: int64

In [12]:
preprocess_url = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"
encoder_url = "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2"

In [13]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

2025-04-19 16:41:33.121068: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


In [36]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [37]:
get_sentence_embeding([
    "500$ discount. Hurry up!",
    "Bhavin, are you up for a volleyball game tommorow!"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[ 0.14455757, -0.4303001 ,  0.9707587 , ..., -0.839485  ,
         0.21332972,  0.98701644],
       [ 0.13080132, -0.5638753 ,  0.9764202 , ..., -0.7293108 ,
         0.05407267,  0.97453773]], dtype=float32)>

In [38]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]],[e[2]])

array([[0.94024074]], dtype=float32)

In [18]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocess_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocess_text)

l = tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1,activation="sigmoid",name="output")(l)

model = tf.keras.Model(inputs=[text_input],outputs=[l])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [19]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
 16/140 [==>...........................] - ETA: 21:46 - loss: 0.5145 - accuracy: 0.7480

KeyboardInterrupt: 

In [22]:
model.evaluate(X_test,y_test)

 1/35 [..............................] - ETA: 6:30 - loss: 0.5239 - accuracy: 0.7812

KeyboardInterrupt: 

In [ ]:
reviews = [
    "You've won the mercedes!"
]
model.predict(reviews)